Data Preparation

In [ ]:
import pandas as pd
import string
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\isaac\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
train_data = pd.read_json('train.json')
test_data = pd.read_json('test.json')

print(train_data.head())
print(test_data.head())

                                             reviews  sentiments
0  I bought this belt for my daughter in-law for ...           1
1  The size was perfect and so was the color.  It...           1
2  Fits and feels good, esp. for doing a swim rac...           1
3  These socks are absolutely the best. I take pi...           1
4  Thank you so much for the speedy delivery they...           1
                                             reviews
0  I bought 2 sleepers.  sleeper had holes in the...
1  I dare say these are just about the sexiest th...
2  everything about the transaction (price, deliv...
3  Not bad for just a shirt.  Very durable, and m...
4  These are truly wrinkle free and longer than t...


In [3]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

In [4]:
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove stop words
    text = ' '.join(word for word in text.split() if word not in stop_words)
    # Remove extra whitespaces
    text = ' '.join(text.split())
    return text

In [5]:
train_data['cleaned_reviews'] = train_data['reviews'].apply(clean_text)
test_data['cleaned_reviews'] = test_data['reviews'].apply(clean_text)

In [6]:
print(train_data[['reviews', 'cleaned_reviews']].head())

                                             reviews  \
0  I bought this belt for my daughter in-law for ...   
1  The size was perfect and so was the color.  It...   
2  Fits and feels good, esp. for doing a swim rac...   
3  These socks are absolutely the best. I take pi...   
4  Thank you so much for the speedy delivery they...   

                                     cleaned_reviews  
0         bought belt daughter inlaw christmas loved  
1            size perfect color looked like web page  
2  fits feels good esp swim race highly recommend...  
3  socks absolutely best take pilates classes hot...  
4  thank much speedy delivery came time rehearsal...  


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['cleaned_reviews'])

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_data['cleaned_reviews'])
test_sequences = tokenizer.texts_to_sequences(test_data['cleaned_reviews'])

# Padding sequences
max_length = max(max(len(seq) for seq in train_sequences), max(len(seq) for seq in test_sequences))
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Display the shape of padded sequences
print(f'Train padded shape: {train_padded.shape}')
print(f'Test padded shape: {test_padded.shape}')

Train padded shape: (7401, 518)
Test padded shape: (1851, 518)


In [8]:
import numpy as np

# Load GloVe word embeddings (download and extract glove.6B.100d.txt)
embedding_dim = 100
glove_file = 'glove.6B.100d.txt'
embeddings_index = {}

with open(glove_file, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print(f'Loaded {len(embeddings_index)} word vectors from GloVe.')


Loaded 400000 word vectors from GloVe.


In [9]:
# Prepare the embedding matrix
vocab_size = len(tokenizer.word_index) + 1  # +1 for padding token
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Map the words in the tokenizer's vocabulary to GloVe vectors
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector  # Words found in GloVe will use pretrained vectors

# Check the embedding matrix shape
print(f'Embedding matrix shape: {embedding_matrix.shape}')


Embedding matrix shape: (16366, 100)
